TRATAMENTO DOS DADOS

In [2]:
import pandas as pd

# 1. Carregar os dados
df_desmatamento = pd.read_csv('estimativa_manaus_mensal.csv', sep=';')
df_mortalidade = pd.read_csv('mortalidade_unificada.csv')
df_queimadas = pd.read_csv('queimadas_unificadas.csv')

# 2. Padronizar nomes
df_desmatamento.columns = ['ANO', 'MES', 'AREA_DESMATADA_KM2']
df_queimadas.columns = df_queimadas.columns.str.upper()

# 3. Transformar mortalidade para formato longo (mantendo CID-10)
meses_map = {
    'Janeiro': 1, 'Fevereiro': 2, 'Março': 3, 'Abril': 4, 'Maio': 5, 'Junho': 6,
    'Julho': 7, 'Agosto': 8, 'Setembro': 9, 'Outubro': 10, 'Novembro': 11, 'Dezembro': 12
}
df_mortalidade_long = df_mortalidade.melt(
    id_vars=['ANO', 'Categoria CID-10'],
    var_name='MES', value_name='OBITOS'
)
df_mortalidade_long['MES'] = df_mortalidade_long['MES'].map(meses_map)

# 4. Agregar queimadas por ano/mês
df_queimadas['MES'] = df_queimadas['MES'].astype(int)
df_queimadas_agg = df_queimadas.groupby(['ANO', 'MES'], as_index=False).agg({
    'FRP': 'mean',
    'RISCOFOGO': 'mean',
    'PRECIPITACAO': 'mean',
    'DIASEMCHUVA': 'mean'
})

# 5. Juntar dados de desmatamento e queimadas (por ano/mês)
df_ambiente = pd.merge(df_desmatamento, df_queimadas_agg, on=['ANO', 'MES'], how='outer')

# 6. Unir com a base de mortalidade (mantendo CID-10 separadamente)
df_final = pd.merge(df_mortalidade_long, df_ambiente, on=['ANO', 'MES'], how='left')

# 7. Salvar resultado
df_final = df_final.sort_values(by=['ANO', 'MES', 'Categoria CID-10'])
df_final.to_csv('dados_unificados_com_cid10.csv', sep=';', index=False)


In [3]:
import pandas as pd

# Carrega os dois arquivos
df_cid10 = pd.read_csv("dados_unificados_com_cid10.csv", sep=";")
df_ar = pd.read_csv("media_mensal_qualidade_ar_manaus.csv", sep=";")

# Garante consistência dos nomes
df_cid10 = df_cid10.rename(columns={"ANO": "ano", "MES": "mes"})

# Mantém apenas as colunas essenciais (ano, mes, pm2.5_atm)
df_ar = df_ar[["ano", "mes", "pm2.5_atm"]]

# Faz o merge por ano e mês
df_merged = pd.merge(df_cid10, df_ar, on=["ano", "mes"], how="left")

# Exibe amostra
print(df_merged.head())

# Salva o resultado
df_merged.to_csv("dados_unificados_com_pm25.csv", sep=";", index=False)


    ano                                   Categoria CID-10  mes OBITOS  \
0  2020                               J01   Sinusite aguda    1      -   
1  2020  J06   Infecc agudas vias aereas super loc mult NE    1      1   
2  2020         J11   Influenza dev virus nao identificado    1      -   
3  2020                         J12   Pneumonia viral NCOP    1      -   
4  2020                        J15   Pneumonia bacter NCOP    1     16   

   AREA_DESMATADA_KM2  FRP  RISCOFOGO  PRECIPITACAO  DIASEMCHUVA  pm2.5_atm  
0            0.070725  NaN        0.0      1.111111     2.333333    10.9135  
1            0.070725  NaN        0.0      1.111111     2.333333    10.9135  
2            0.070725  NaN        0.0      1.111111     2.333333    10.9135  
3            0.070725  NaN        0.0      1.111111     2.333333    10.9135  
4            0.070725  NaN        0.0      1.111111     2.333333    10.9135  


In [4]:
import pandas as pd
import glob

# --- Mortalidade ---
print("📄 Checando mortalidade...")
mortalidade_path = 'base_dados/mortalidade'
for file in glob.glob(f'{mortalidade_path}/mortalidade_*.csv'):
    year = int(file[-8:-4])
    print(f"\n➡ Lendo arquivo: {file} (ano {year})")
    df = pd.read_csv(file, sep=';', encoding='latin1')
    print(f"Colunas encontradas: {list(df.columns)}")


📄 Checando mortalidade...


In [5]:
import pandas as pd

# Carrega a tabela unificada gerada anteriormente
df = pd.read_csv("dados_unificados_com_pm25.csv", sep=";")

# Função para classificar qualidade do ar com base no PM2.5
def classificar_qualidade_ar(pm25):
    if pd.isna(pm25):
        return "Sem dados"
    elif pm25 <= 12.0:
        return "Boa"
    elif pm25 <= 35.4:
        return "Moderada"
    elif pm25 <= 55.4:
        return "Ruim"
    elif pm25 <= 150.4:
        return "Muito Ruim"
    elif pm25 <= 250.4:
        return "Péssima"
    else:
        return "Perigosa"

# Aplica a função
df["QUALIDADE_AR_CLASSIFICADA"] = df["pm2.5_atm"].apply(classificar_qualidade_ar)

# Exibe exemplo
print(df[["ano", "mes", "pm2.5_atm", "QUALIDADE_AR_CLASSIFICADA"]].head())

# Salva resultado
df.to_csv("dados_unificados_com_classificacao_ar.csv", sep=";", index=False)


    ano  mes  pm2.5_atm QUALIDADE_AR_CLASSIFICADA
0  2020    1    10.9135                       Boa
1  2020    1    10.9135                       Boa
2  2020    1    10.9135                       Boa
3  2020    1    10.9135                       Boa
4  2020    1    10.9135                       Boa
